In [ ]:
from tradingstrategy.client import Client

client = Client.create_jupyter_client()

Started Trading Strategy in Jupyter notebook environment, configuration is stored in /Users/moo/.tradingstrategy


In [ ]:

from tradeexecutor.monkeypatch.dataclasses_json import patch_dataclasses_json
from tradeexecutor.state.state import State
import requests

# Currently needed because unpatched dataclasses_json package issues
patch_dataclasses_json()

# Public internet endpoint as exposed by the trade executor Docker
webbhook_url = "https://enzyme-polygon-multipair.tradingstrategy.ai/"

state_api = f"{webbhook_url}/state"

resp = requests.get(state_api)

state_blob = resp.content

print(f"Downloaded {len(state_blob):,} bytes state data")

state = State.read_json_blob(state_blob)

print(f"trade-executor was launched at: {state.created_at}, we have {len(list(state.portfolio.get_all_trades()))} trades")

Downloaded 10,344,843 bytes state data


/Users/moo/Library/Caches/pypoetry/virtualenvs/trade-executor-8Oz1GdY1-py3.10/lib/python3.10/site-packages/dataclasses_json/core.py:187: RuntimeWarning: 'NoneType' object value of non-optional type initial_cash detected when decoding TradeSummary.
  warnings.warn(


trade-executor was launched at: 2023-06-23 13:01:41, we have 174 trades


In [ ]:
from io import StringIO
from typing import Iterable, List
import pandas as pd

from tradeexecutor.state.position import TradingPosition
from tradeexecutor.state import position 

positions = state.portfolio.closed_positions.values()

data = [(p.closed_at, p.get_realised_profit_percent() * 100, p.is_repaired) for p in positions]

df = pd.DataFrame(data)
df

,0,1
0,2023-06-29 13:31:00,0.000000
1,2023-06-30 05:36:10,0.702420
2,2023-07-03 07:48:02,0.000000
3,2023-07-03 07:48:02,0.000000
4,2023-07-03 07:48:02,0.000000
...,...,...
82,2023-10-23 23:09:31,3.416912
83,2023-10-24 03:18:31,0.994867
84,2023-10-24 03:33:28,3.230938
85,2023-10-26 08:32:10,0.000000


### Breakpoint
Convert to normal for loop (with enumerate) and set conditional breakpoint here (i == 80). After reaching this breakpoint, the next step is to step into the `get_size_relative_realised_profit_percent` method and go from there.

```python

https://github.com/tradingstrategy-ai/trade-executor/blob/7c74a2ce9443558f7c40abf6675a57bf05a78108/tradeexecutor/visual/equity_curve.py#L366

```python
data = []
for i, p in enumerate(state.portfolio.closed_positions.values()):
    if not p.is_closed():
        continue

    data.append((p.closed_at, p.get_size_relative_realised_profit_percent()))
```

In [ ]:
from tradeexecutor.visual.equity_curve import calculate_compounding_realised_trading_profitability

profitability = calculate_compounding_realised_trading_profitability(state)

returns_all_time = profitability.iloc[-1]

print(f"Returns all time: {returns_all_time:.2%}")

Returns all time: -29.06%
